In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px
# import plotly.graph_objects as go
# import plotly.figure_factory as ff
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
# from nltk.tokenize import word_tokenize as wt 
# from collections import Counter
import gensim
# import textblob
import spacy
import tensorflow as tf
import pickle
import string
import re
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM, Bidirectional

In [2]:
vectorizer = pickle.load(open('../../../../../../word2vec_files/glove_twitter_200.pkl', "rb"))
model = tf.keras.models.load_model("../saved_model/saved_bi_lstm_model")
loaded_tokenizer = pickle.load(open("../saved_model/tokeniser.pkl",'rb'))

In [3]:
from spacy.lang.en.stop_words import STOP_WORDS

# print(STOP_WORDS,'\n') # <- set of Spacy's default stop words
all_stopwords = STOP_WORDS
# Adding several stopwords
# all_stopwords |= {'not',"no", "n't", 'n’t','n‘t','cannot','none','without','against'}

# Removing several stop words
all_stopwords-= {'not',"no", "n't", 'n’t','n‘t','cannot','none','without','against','off','against','too'}
my_stop_words = set(all_stopwords) # My own stop words

In [4]:
nlp = spacy.load("en_core_web_sm")

In [98]:
give_txt = input("Slot in your text > ")
# give_txt = "I really dislike the cost of this product"

In [99]:
# https://stackoverflow.com/questions/12851791/removing-numbers-from-string
def preprocess(sent):
    '''Cleans text data up, leaving only 2 or
        more char long non-stepwords composed of A-Z & a-z only
        in lowercase'''
    # lowercase
    sentence = sent.lower()

    # Remove RT
    sentence = re.sub('RT @\w+: '," ",sentence)

    # Remove special characters
    sentence = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", sentence)

    # Removing digits
    sentence = sentence.translate(str.maketrans('', '', string.digits))

    # Removing puntuactions
    # sentence = sentence.translate(str.maketrans('', '', string.punctuation))

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  
    # When we remove apostrophe from the word "Mark's", 
    # the apostrophe is replaced by an empty space. 
    # Hence, we are left with single character "s" that we are removing here.

    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  
    # Next, we remove all the single characters and replace it by a space 
    # which creates multiple spaces in our text. 
    # Finally, we remove the multiple spaces from our text as well.

    return sentence


In [100]:
preprocessed_text = preprocess(give_txt)

In [101]:
def spacy_tokeniser(sent):
    sent = sent.strip().lower()
    doc = nlp(sent)
    mytokens = [token.lemma_ for token in doc if token.text not in my_stop_words]
    return mytokens

In [102]:
lemmatised_text = spacy_tokeniser(preprocessed_text)

In [103]:
X = [lemmatised_text]

In [104]:
X = loaded_tokenizer.texts_to_sequences(X)

In [105]:
max_len = 300
X = pad_sequences(X,padding="post",maxlen=max_len)

In [106]:
X[0].shape # shape (300,)
# X[1].ndim # dimension

(300,)

In [107]:
y_pred = model.predict(X)

1/1 [==============================] - 0s 87ms/step


In [108]:
prediction = np.argmax(y_pred, axis = 1)
print(f"This is your given review:\n{give_txt}")
if prediction == 0:
    print("Negative review")
elif prediction == 1:
    print("Neutral review")
elif prediction == 2:
    print("Positive review")

This is your given review:
I dislike purchasing products from EBAY, it is always expensive and ends up being useless and worthless😒
Negative review
